In [1]:
from app.agent.multi_agent import MultiAgent
from app.agent.agent import ContextTypeAgent, QuestionTypeAgent
from app.prompts.prompts import SIMPLE_VS_COMPLEX, GENERAL_VS_PARTICULAR_CONTEXT
from app.prompts.prompt import Prompt


simple_vs_complex_prompt = Prompt(prompt=SIMPLE_VS_COMPLEX)
simple_vs_complex_agent = QuestionTypeAgent(
    instruction=simple_vs_complex_prompt
)

general_vs_particular_prompt = Prompt(GENERAL_VS_PARTICULAR_CONTEXT)
general_vs_particular_agent = ContextTypeAgent(
    instruction=general_vs_particular_prompt
    )

multi_agent = MultiAgent(agents=[
    simple_vs_complex_agent, 
    general_vs_particular_agent
])

/Users/Jaime/Desktop/Project-Ideas/python-gpt/.venv/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
import asyncio
import nest_asyncio

nest_asyncio.apply()

async def main(**kwargs):
    return await multi_agent.pipeline(**kwargs)

tool, output = asyncio.run(main(query="What will happen If I remove the line 'os.remove(name_file)' of the function _create_node_relationships_file"))
print(output.subject)

Agent reasoning response: The subject is the function _create_node_relationships_file therefore the answer is complex because there is only one subject but it is a function
Agent answer: complex 

Not valid answer. Applying retries
	Retry agent response: {"question_type": "simple", "subject": ["_create_node_relationships_file"], "reasoning": "The subject is the function _create_node_relationships_file therefore the answer is simple since there is only one subject"}
Agent reasoning response: The question refers to what will happen if you remove a line of code, which is not specific to the subject itself, but rather its usage in the broader context.
Agent answer: general 

Tool decided by the agent: GeneralRetriever
{'_create_node_relationships_file'}


In [3]:
tool = asyncio.run(
    main(query="What does the method create_tools_from_defaults from the class Tools do?")
)

Agent reasoning response: The subjects are: create_tools_from_defaults method and Tools class, therefore the answer is simple because there is only one method and one class as subjects.
Agent answer: simple 

Not valid answer. Applying retries
	Retry agent response: {"question_type": "simple", "subject": ["create_-tools_from_defaults"], "reasoning": "The subject is the method create_tools_from_defaults therefore the answer is simple because there is only one subject"}
Agent reasoning response: The question is regarding the method itself, therefore the answer is particular
Agent answer: particular 

Tool decided by the agent: SimilarityRetriever


In [4]:
tool = asyncio.run(
    main(query="What are the main differences between the retriever RetrieverA and the retriever RetrieverB?")
)

Agent reasoning response: The subjects are: RetrieverA and RetrieverB classes or functions, therefore the answer is complex because there are more than one subject.
Agent answer: complex 

Tool decided by the agent: SimilarityRetriever


In [5]:
tool = asyncio.run(main(
    query = "What is the difference between the method __retrieve_relationship_nodes and the method _check_relationships_of_retrieved_nodes"
))

Agent reasoning response: The subjects are the methods __retrieve_relationship_nodes and _check_relationships_of_retrieved_nodes, therefore the answer is complex because there are more than one subject.
Agent answer: complex 

Tool decided by the agent: SimilarityRetriever


# Integration with database functionality

In [6]:
from app.database.base import get_db
from sqlalchemy.orm import Session

db = next(get_db())

In [7]:
from app.retrievers.general_retriever import GeneralRetriever
from app.retrievers.similarity_retriever import SimilarityRetriever

async def tool_pipeline(agent: MultiAgent, query: str, db: Session=db):
    tool, output = await agent.pipeline(query=query)
    tool: GeneralRetriever | SimilarityRetriever = tool(db=db)
    return tool.query_database(query=query, subjects=output.subject)

In [8]:
query = "What will happen If I remove the line 'os.remove(name_file)' of the function _create_node_relationships_file"
nodes = asyncio.run(tool_pipeline(agent=multi_agent, query=query, db=db))

Agent reasoning response: The subject is the function _create_node_relationships_file therefore the answer is simple because there is only one subject.
Agent answer: simple 

Agent reasoning response: The question refers to the action of removing a line, which affects something outside the subject, therefore the answer is general
Agent answer: general 

Tool decided by the agent: GeneralRetriever


/Users/Jaime/Desktop/Project-Ideas/python-gpt/.venv/lib/python3.12/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


Metadata of the node that we're considering: {'arguments': 'db: Session', 'function_name': '_create_node_relationships_file'}
The chosen node is a function node
	{'hash': '4496633ed598fb2c5ae7412cda2fe78abdc877073c24d8e215f96a6e32ba0f6d', 'lines_of_code': [332, 414], 'additional_metadata': {'arguments': 'db: Session', 'function_name': '_create_node_relationships_file'}}


In [9]:
for node in nodes:
    print(node.text)

def _create_node_relationships_file(db: Session):
    
    fields = ('class_name', 'function_name', 'method_name')
    name_file = os.environ['NAMES_FILE']
    relationships_file = os.environ['RELATIONSHIPS_FILE']
    
    os.remove(name_file)
    
    if not os.path.exists(name_file):
        nodes = []
        for field in fields:
            nodes_metadata = db.query(NodeMetadata)\
                .filter(NodeMetadata.node_metadata[field].astext != None).all()
                
            nodes.extend(nodes_metadata)
            
            with open(name_file, "w") as names_file:
                for node in nodes:
                    node_metadata: NodeMetadata = node.node_metadata
                    for field in fields:
                        name = node_metadata.get(field)
                        if name is None or name == "__init__": continue
                        names_file.write(f"{name} {node.node_id} \n")
    
    command = f"./bash-scripts/find-node-relationships.sh {n

In [13]:
query = "Explain to me how the class NodePostProccesor is implemented"
nodes = asyncio.run(tool_pipeline(agent=multi_agent, query=query, db=db))

Agent reasoning response: The subject is the class NodePostProccesor therefore the answer is complex because there is only one subject
Agent answer: complex 

Not valid answer. Applying retries
	Retry agent response: {"question_type": "simple", "subject": ["NodePostProcessor"], "reasoning": "The subject is the class NodePostProcessor therefore the answer is simple because there is only one subject"}
Agent reasoning response: The question is regarding the implementation of the subject itself, therefore the answer is particular
Agent answer: particular 

Tool decided by the agent: SimilarityRetriever
Node obtainer with score: 0.8044066668870056 --> class NodePostProccesor:
  
    def __init__(self, retrieved_nodes_score: List[NodeWithScore], db: Session, score_threshold: float=0.8, min_parent_nodes: int=2, min_file_nodes: int=2):
        
        self._retrieved_nodes_score = [node for node in retrieved_nodes_score if node.score > score_threshold]
        self._retrieved_nodes = [node.no

In [ ]:
for node in nodes:
    print(node.text)